In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString
from shapely.ops import split

In [2]:
#Read in routes feature class as shapefile
gdfRoutes = gpd.read_file('..\\data\\processed\\Routes.shp')

In [3]:
#Extract start and end points from each route
startPoints = gdfRoutes['geometry'].apply(lambda x: Point(x.coords[0]))
endPoints = gdfRoutes['geometry'].apply(lambda x: Point(x.coords[-1]))

In [4]:
#Copy routes geodataframe and update geometry to start points
gdfStart = gdfRoutes.copy(deep=True)
gdfStart['geometry'] =startPoints
gdfStart.to_file('../data/processed/starts.shp')

In [5]:
#Copy routes geodataframe and update geometry to start points
gdfEnd = gdfRoutes.copy(deep=True)
gdfEnd['geometry'] = endPoints
gdfEnd.to_file('../data/processed/ends.shp')

Create junctions along route segments where upstream routes join them - done by iterating through each route feature and splitting it by the set of end points. 

In [6]:
#Combine endpoints into a single multipoint object
ends = endPoints.unary_union

In [7]:
#Create a geoseries of split routes (geometry collections)
theSplits = gdfRoutes.geometry.apply(lambda x: split(x,ends))

In [27]:
gdfRoutes.dtypes

level_0          int64
index            int64
Facility N      object
Address         object
City            object
County Nam      object
Zip            float64
Latitude       float64
Longitude      float64
Regulated       object
Allowable        int64
Total Wast     float64
Biogas Pot     float64
Biogas P_1     float64
geometry      geometry
dtype: object

In [36]:
#Create lists to fill
links = []
biogas = []
geom = []
#Iterate and add items to the list
for index, row in gdfRoutes.iterrows():
    #Iterate through split segments in the geometry collection
    for line in theSplits[index].geoms:
        #Add items to the list
        links.append(row['index'])
        biogas.append(row['Biogas P_1'])
        geom.append(line)
#Construct a new dataframe

#Construct an output geodataframe
gdfSegments = gpd.GeoDataFrame(pd.DataFrame({'link':links,'biogas':biogas}),
                               geometry = geom, crs = gdfRoutes.crs)
    
    

In [51]:
#Add from node and to node to gdfSegments
gdfSegments2 = gpd.sjoin(left_df=gdfSegments,
                         right_df=gdfStart[['index','geometry']],
                         how='left',rsuffix='start')

In [52]:
gdfSegments2.head()

,link,biogas,geometry,index_start,index
0,179,2.456923e+08,"LINESTRING (1582814.308 -311794.717, 1582814.3...",0.0,179.0
1,179,2.456923e+08,"LINESTRING (1582309.906 -310281.512, 1582309.9...",NaN,NaN
2,179,2.456923e+08,"LINESTRING (1582309.906 -308768.307, 1582309.9...",NaN,NaN
3,179,2.456923e+08,"LINESTRING (1582309.906 -308263.906, 1582309.9...",NaN,NaN
4,179,2.456923e+08,"LINESTRING (1582309.906 -307255.102, 1582814.3...",NaN,NaN


In [50]:
gdfSegments3 = gpd.sjoin(left_df=gdfSegments2,
                         right_df=gdfEnd[['index','geometry']],
                         how='left',rsuffix='end')

ValueError: Index data must be 1-dimensional

In [49]:
gdfSegments3.to_file('../data/processed/LineSegements_StartEnd.shp')

ValueError: Index data must be 1-dimensional

In [38]:
gdfSegments.to_file('../data/processed/LineSegements.shp')

In [48]:
gdfSegments2.to_file('../data/processed/LineSegements_Start.shp')

In [35]:
gdfRoutes.columns

Index(['level_0', 'index', 'Facility N', 'Address', 'City', 'County Nam',
       'Zip', 'Latitude', 'Longitude', 'Regulated', 'Allowable', 'Total Wast',
       'Biogas Pot', 'Biogas P_1', 'geometry'],
      dtype='object')